In [27]:
import numpy as np
import cv2
import matplotlib as plt
import time, csv, os, shutil, sys



j=0
label=0
trainPath = "../Mood_Detector/training.csv"
dstPath = "../Mood_Detector/"
imgPath = "../Mood_Detector/Manually_Annotated/Manually_Annotated_Images/"

with open(trainPath, 'r') as trainFile:
    sp = []
    skip_to_line = 0
    for lines in trainFile:
        skip_to_line += 1
        # skip header
        if skip_to_line == 1:
            for lines in trainFile:
                sp = lines.split(',')
                # get the label
                label = sp[6]
                # move the image file path into corresponding target directory
                shutil.move(imgPath+sp[0],dstPath + label + "/")

print("done")


done


In [27]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib as plt
import time, csv, os, shutil, sys
from IPython.display import display, clear_output


trainPath = "../Mood_Detector/training.csv"
dstPath = "../Mood_Detector/"
imgPath = "../Mood_Detector/Manually_Annotated/Manually_Annotated_Images/"

CATEGORIES = ["Neutral","Happiness","Sadness","Surprise","Fear","Disgust","Anger","Contempt"]
training_data = []
IMG_SIZE = 64
i=0


def createTrainData():    
    for category in CATEGORIES:
        path = os.path.join(dstPath, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
    

createTrainData()
print("done")

done


In [38]:
import random 

random.shuffle(training_data)

X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)


In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [1]:
import pickle
import numpy as np

IMG_SIZE = 64

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [2]:
print(X.shape)

(63942, 64, 64, 3)


In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [4]:
def getHalfData(data):
    d1 = data[:int(len(data)/2)]
    return d1

# i'm Normalizing half the data
def dataNormalization(data):
    d1 = data[:int(len(data)/2),:,:,:].astype('float32')
    d1 /= 255
#     d2 = data[int(57547/2):,:,:,:].astype('float16')
#     d2 /= 255
#     normalized = np.vstack((d1,d2))
#     return normalized
    return d1
    
x_train = dataNormalization(x_train)
x_test = dataNormalization(x_test)
y_train = getHalfData(y_train)
y_test = getHalfData(y_test)


print("done")

done


In [5]:
print(x_train.shape)
print(x_test.shape)
print(len(y_train))
print(len(y_test))
# print(y_train)
# print(y_test)

(28773, 64, 64, 3)
(3197, 64, 64, 3)
28773
3197


In [6]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, Adadelta
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, TensorBoard

# num_classes = 10
num_classes = 8
num_epochs = 40
num_batch = 32

# # train data and test data for cifar10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print(x_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
# input : 64, 64 color RGB
# conv1
model.add(Conv2D(66, (3,3), padding='same', input_shape=x_train.shape[1:])) # change input shape
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))

# Conv 2, 3 and 4
model.add(Conv2D(128,(3,3),padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
# Activation
model.add(Conv2D(128,(3,3)))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
# Activation
model.add(Conv2D(128,(3,3)))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))

# Conv 5
model.add(Conv2D(192, (3,3)))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# maxpooling2d
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))

# Conv 6, 7, 8 and 9
model.add(Conv2D(192,(3,3),padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
model.add(Conv2D(192,(3,3),padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))


# Conv 9
model.add(Conv2D(288, (3,3), padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
# maxpooling2d
model.add(MaxPooling2D(pool_size=(2,2)))


# Conv 10
model.add(Conv2D(288, (3,3), padding='same'))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))

# Conv 11
model.add(Conv2D(355, (3,3)))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))

# Conv 12
model.add(Conv2D(432, (3,3)))
# BatchNormalisation
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001, scale=True))
# Activation
model.add(Activation('relu'))
# DropOut 
model.add(Dropout(0.2))
# global max pooling
model.add(GlobalMaxPooling2D())

# fully connected Layer MLP 
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax', bias_initializer='constant'))


# Initialise the optimizer Adam 
#optim = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optim = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)


# Compile the model 
model.compile(loss = 'categorical_crossentropy', optimizer=optim, metrics = ['accuracy'])

# model.summary()

# Normalize the data
# x_train = x_train.astype('float32')
# x_train /= 255.0
# x_test = x_test.astype('float32')
# x_test /= 255.0



# If we are using data augmentation then we augment the images
#horizontal/vertical flip and random rotation
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True)

# apply the transformation on the train images
datagen.fit(x_train)

# we want to reduce the LR once we hit a plateau 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, verbose=1, mode='auto')


# Fit the model 
model.fit(x_train, y_train, batch_size=num_batch, epochs=num_epochs, callbacks=[reduce_lr], validation_split=0.2, shuffle=True )

# Then we evaluate the model with evaluate()
# and we print the model
score = model.evaluate(x_test, y_test, batch_size=num_batch)
print(score)



C:\Users\Nadir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 23018 samples, validate on 5755 samples
Epoch 1/40
23018/23018 [==============================] - 163s 7ms/step - loss: 2.4607 - acc: 0.1537 - val_loss: 2.0282 - val_acc: 0.1491
Epoch 2/40
23018/23018 [==============================] - 155s 7ms/step - loss: 2.2181 - acc: 0.1687 - val_loss: 2.0289 - val_acc: 0.1482
Epoch 3/40
23018/23018 [==============================] - 155s 7ms/step - loss: 2.1460 - acc: 0.1806 - val_loss: 2.0424 - val_acc: 0.1480
Epoch 4/40
23018/23018 [==============================] - 155s 7ms/step - loss: 2.1001 - acc: 0.1901 - val_loss: 2.0255 - val_acc: 0.2083
Epoch 5/40
23018/23018 [==============================] - 155s 7ms/step - loss: 2.0396 - acc: 0.2188 - val_loss: 2.0950 - val_acc: 0.2087
Epoch 6/40
23018/23018 [==============================] - 155s 7ms/step - loss: 1.8758 - acc: 0.3095 - val_loss: 1.8630 - val_acc: 0.3051
Epoch 7/40
23018/23018 [==============================] - 155s 7ms/step - loss: 1.7566 - acc: 0.3503 - val_loss: 1.8111 - v

In [7]:
model.save("MoodSimpNetV2-1DenseLayer-Softmax40.h5")

In [1]:
import face_recognition
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

nameList = []
CATEGORIES = ["Neutral","Happiness","Sadness","Surprise","Fear","Disgust","Anger","Contempt"]


IMG_SIZE = 64

# Load the pre-trained model
model = load_model('MoodSimpNetV2-1DenseLayer-Softmax40.h5')


# Get a reference to webcam #0
video_capture = cv2.VideoCapture(0)




while True:
    
    #get a single frame of video
    ret, frame = video_capture.read()
    
    # resizing the frame for faster face recognition processing
    faceFrame  = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)
    
    # we take the face location(s) in the image 
    face_locations = face_recognition.face_locations(frame)


    i = 0

    # Notes: instead of saving the images we could 
    # pass the image straight to the CNN for classification
    # but we will have to 
    for face_location in face_locations:
        
        
        # carré 129 par 129
        top, right, bottom, left = face_location

#         print(frame)
        # we create the image and scale it down to 64 x 64 
        cropped_img = frame[left-100:right+10, top-50:bottom+50]
        plt.imshow(cropped_img)
        new_array = cv2.resize(cropped_img,(IMG_SIZE, IMG_SIZE))
        new_array = cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB)
        new_array = new_array[np.newaxis,:,:,:]
        new_array = new_array.astype('float32')
        new_array /= 255


        
        # we then classify the image and print the result
        moodPrediction = model.predict(new_array)
        print(CATEGORIES[np.argmax(moodPrediction)])
        
        # we draw rectangles around the faces
#         cv2.rectangle(frame,(left-50,top-50), (right+50, bottom+50), (0,0,255), 2)
        # we draw the labels with the names over, or below the face
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    # we Display the image
    cv2.imshow('Video',frame)

        # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    i += 1




# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()



C:\Users\Nadir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Sadness
Sadness
Happiness
Contempt
Neutral
Neutral
Neutral
Happiness
Contempt
Happiness
Neutral
Neutral
Neutral
Neutral
Contempt
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Contempt
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Happiness
Sadness
Happiness
Happiness
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Sadness
Sadness
Sadness
Neutral
Contempt
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness
Sadness

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

# Load the pre-trained model
model = load_model('MoodSimpNetV2-1DenseLayer-Softmax40.h5')

IMG_SIZE = 64

img_array = cv2.imread('../Mood_Detector/Happiness/00fc0b2aab0d50be8cfe8793c65dfe019cf7ed6b04ec124d3f21f7b4.JPG')
img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
new_array = new_array[np.newaxis,:,:,:]
new_array = new_array.astype('float32')
new_array /= 255


new_array = model.predict(new_array)


In [16]:
np.argmax(new_array)

[[6.4451662e-08 9.9867320e-01 1.3431643e-08 2.3498749e-06 8.2302920e-09
  1.8177511e-07 1.9980975e-08 1.3241724e-03]]


1